# **SaprotHub: Making Protein Modeling Accessible to All Biologists**

<a href="https://www.biorxiv.org/content/10.1101/2024.05.24.595648v3"><img src="https://img.shields.io/badge/Paper-bioRxiv-green" style="max-width: 100%;"></a>
<a href="https://huggingface.co/SaProtHub"><img src="https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-red?label=SaprotHub" style="max-width: 100%;"></a>
<a href="https://github.com/westlake-repl/SaprotHub"><img src="https://img.shields.io/badge/Github-black?logo=github" style="max-width: 100%;"></a>
<a href="https://theopmc.github.io/"><img src="https://img.shields.io/badge/Website-OPMC-yellow" style="max-width: 100%;"></a>
<a href="https://cbirt.net/no-coding-required-saprothub-brings-protein-modeling-to-every-biologist/" alt="blog"><img src="https://img.shields.io/badge/Blog-Medium-purple" /></a>
<a href="https://x.com/sokrypton/status/1795525127653986415"><img src="https://img.shields.io/badge/Twitter-blue?logo=twitter" style="max-width: 100%;"></a>


This is **ColabSaprot**, the Colab version of [SaProt](https://github.com/westlake-repl/SaProt), a pre-trained protein language model designed for various downstream protein tasks.

**ColabSaprot** is a platform where **Protein Language Models(PLMs)** are more accessible and user-friendly for biologists, enabling effortless model training and sharing within the scientific community.

We've established the **SaprotHub**([website](https://huggingface.co/SaProtHub), [paper](https://www.biorxiv.org/content/10.1101/2024.05.24.595648v3)) for storing and sharing models and datasets, where you can explore extensive collections for specific protein prediction tasks.

We hope ColabSaprot and SaprotHub can contribute to advancing biological research, fostering collaboration, and accelerating discoveries in the field. You can access [our paper](https://www.biorxiv.org/content/10.1101/2024.05.24.595648v3) for further details.

Check these videos ([training](https://www.youtube.com/watch?v=r42z1hvYKfw), [predicting](https://www.youtube.com/watch?v=N5VMBwM_ukQ)) to see how to use  ColabSaprot.

Joining [**OPMC**](https://theopmc.github.io/) as an author of SaprotHub.

ColabSaprot supports hundreds of [protein prediction tasks](https://github.com/westlake-repl/SaProtHub/blob/main/task_list.md).







In [2]:
#@title **Click the run button to find the most suitbale module for your task**


from IPython.display import display
import ipywidgets


def clicked(button):

  print(button.description)
  button.description = "wwwww"
  print("button has been clicked!")

button_download = ipywidgets.Button(description = 'Test Button')
button_download.on_click(clicked)
display(button_download)

Button(description='Test Button', style=ButtonStyle())

In [3]:
#######################################################################
#           Training or Inference            #
#######################################################################
display(ipywidgets.Label(value="The $m$ in $E=mc^2$:"), button_download)
sss = 1

Label(value='The $m$ in $E=mc^2$:')

Button(description='wwwww', style=ButtonStyle())